60. 単語ベクトルの読み込みと表示

Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

In [ ]:
from gensim.models import KeyedVectors
import numpy as np

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

a = model["United_States"]

a.shape

61. 単語の類似度

“United States”と”U.S.”のコサイン類似度を計算せよ．

In [ ]:
b = model["U.S."]
cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)) 
cos_sim

62. 類似度の高い単語10件

“United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

In [ ]:
model.most_similar("Hello",topn=10)

63. 加法構成性によるアナロジー

“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [ ]:
spain = model["Spain"]
madrid = model["Madrid"]
athens = model["Athens"]
res = spain - madrid + athens
model.most_similar(res,topn=10)
model.most_similar(positive=['Spain','Athens'], negative=['Madrid'],topn=10)

64. アナロジーデータでの実験

単語アナロジーの[評価データ](http://download.tensorflow.org/data/questions-words.txt)をダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

In [ ]:
import os
with open('questions-words.txt',"r") as reads:
    sqs = [s.rstrip() for s in reads.readlines()]
with open('sep.txt',"w") as writes:
    length = len(sqs)
    for i,col in enumerate(sqs):
        q = col.split(' ')
        if len(q) == 4:
            best = model.most_similar(positive=[q[1],q[2]], negative=[q[0]])[0]
            q += [best[0], str(best[1])]
            output = ' '.join(q)+'\n'
        else:
            output = col + '\n'
        writes.write(output)
        os.system('clear')
        reach = i*100/20000
        if reach == round(reach):
            print(str(reach)+'%')

65. アナロジータスクでの正解率

64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

In [ ]:
with open('sep.txt',"r") as f:
    semantic, syntactic = [], []
    for line in f:
        if line.startswith(": gram"):
            ctg =  "syntactic"
        elif line.startswith(":"):
            ctg =  "semantic"
        else:
            if ctg == "semantic":
                semantic.append(line.rstrip("\n").split())
            if ctg == "syntactic":
                syntactic.append(line.rstrip("\n").split())

def calc(data):
  """data: List[List[str]{5}]"""
  ndarray = np.array(data)
  ans, prd = ndarray[:, 3:5].T
  return np.mean(ans==prd)

print("semantic: ", calc(semantic))
print("syntactic: ", calc(syntactic))

66. WordSimilarity-353での評価

[The WordSimilarity-353 Test Collection](http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.html)の評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

In [ ]:
from scipy.stats import spearmanr

sim_rank, human_rank = [], []
with open('wordsim353/combined.csv',"r") as f:
    for line in list(f)[1:]:
        word1,word2,human = line.replace('\n','').split(",")
        if word1 in model and word2 in model:
            x = model.similarity(word1,word2)
            sim_rank.append(x)
            human_rank.append(human)

correlation, pvalue = spearmanr(sim_rank, human_rank)
print(correlation, pvalue)


67. k-meansクラスタリング

国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．

In [ ]:
country = {}
show_list = []
with open('questions-words.txt',"r") as f:
    for col in f:
        if col.startswith(": currency"):
            break
        elif col.startswith(":"):
            continue
        else:
            _,w,_,c=col.replace("\n","").split()
            if c not in country and c in model:
                country[c] = model[c]
                show_list.append(model[c])
            if w not in country and w in model:
                country[w] = model[w]
                show_list.append(model[w])

from sklearn.cluster import KMeans

kmeans_model_predict = KMeans(n_clusters=5, random_state=10, n_init='auto').fit_predict(list(country.values()))
print([list(country.keys())[i] for i, p in enumerate(kmeans_model_predict) if p == 1])

from collections import Counter
c = Counter(kmeans_model_predict)
print(c)

68. Ward法によるクラスタリング

国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram
import matplotlib.pyplot as plt

plt.figure(figsize=(36, 12))
Z = linkage(list(country.values()), method='ward')
dendrogram(Z, labels=list(country.keys()))
plt.xticks(fontsize=14)
plt.show()

69. t-SNE による可視化

ベクトル空間上の国名に関する単語ベクトルをt-SNEで可視化せよ．

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
X = tsne.fit_transform(np.array(show_list))

plt.scatter(X[:,0], X[:,1])
plt.colorbar()
plt.show()